---
# Pupil constriction/dilation
---

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
from remove_outliers import RM_Outliers

In [2]:
%matplotlib auto

Using matplotlib backend: MacOSX


## Read xml Patient File
look for '< ICSVideo >'
xmlns="http://tempuri.org/PMRExportDataSet.xsd"

In [3]:
from video_dat import VideoDat, time_stamp

In [26]:
if 0:
    vid_dat = VideoDat('../data/nico_B_2020_01_17_11_03_03.xml')
    new_vid_dat = [x for x in vid_dat if x['cond']]
    print([f'{i}.'+x['cond'].split('<')[0] for i, x in enumerate(new_vid_dat)])
else:
#     vid_dat = VideoDat('../data/YY_ZZ_2020_01_21_15_42_00.xml')
    vid_dat = VideoDat('../data/YY_ZZ_2020_01_23_14_28_46.xml')
    new_vid_dat = [x for x in vid_dat if x['cond']=='AC2Hz']


## Setting up video file

In [33]:
_id = 2
fname = new_vid_dat[_id]['filename']
start_time = time_stamp(new_vid_dat[_id]['start_t'])
end_time = time_stamp(new_vid_dat[_id]['end_t'])
modified_time = time_stamp(new_vid_dat[_id]['mod_t'])
cond = new_vid_dat[_id]['cond']

print('../videos/'+fname)
print(time.ctime(start_time))
print(cond)

../videos/YY_ZZ_2020_01_23_14_25_38_VO.avi
Thu Jan 23 14:25:38 2020
AC2Hz


In [34]:
thresh=52
index = -1
Area_raw =[]
tSTP = []

cap = cv2.VideoCapture('../videos/'+fname)
# first_frame_time = cap.get(cv2.CAP_PROP_POS_MSEC)

while True:
    read_flag, frame = cap.read()
    
    if not read_flag:
        break
    
    
    gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    FR = gray.copy()
    
    ## Pupil detection
    _, pupil = cv2.threshold(gray,thresh,255,cv2.THRESH_BINARY_INV)
    contoursP, _ = cv2.findContours(pupil, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    for c in contoursP:
        
        if len(c) < 4:
            continue
        
        area = cv2.contourArea(c)
        
        if area<150:
            continue
        
        peri = cv2.arcLength(c, True)
        M = cv2.moments(c)# check moments of an image for centroids calaculation
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        cv2.drawContours(FR,c,index,155)
        cv2.drawMarker(FR,(cx,cy),100)
        Area_raw.append(area)
        tSTP.append(cap.get(cv2.CAP_PROP_POS_MSEC))
    
    if False:
#         cv2.imshow('Gray', gray)
        cv2.imshow('Pupil', FR)
    
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

In [35]:
t = np.array(tSTP)/1000
area = np.array(Area_raw)
res_area = area-area[t<10].mean() # resset over first 10 second : prior to stimulation
new_area = RM_Outliers(res_area,10) # remove outliers (blinks)

lab = str(time.ctime(start_time))
lab = cond


plt.figure()
plt.plot(t,res_area, label=lab ,color='k')
plt.plot(t,new_area,color='r')
plt.legend()

## averaging 3 trials

In [17]:
# AREA=[]
AREA.append(new_area[:3600])

In [18]:
SHAM = np.nanmean(AREA,0)

## filtering

In [ ]:
from scipy import signal

In [ ]:
fs = 120 # sampling frequency
fc = 5 # cut-off frequency
w = fc / (fs / 2) # Normalize the frequency
b, a = signal.butter(4, w, 'low')
output = signal.filtfilt(b, a, new_area)

In [ ]:
plt.figure(999)
plt.plot(t,output, label=lab ,color='k')
plt.legend()

# Adding Stimulation Time Stamp

In [ ]:
StarStim_time = 1579275340743/1000
offset = StarStim_time-start_time
plt.axvline(x=offset-1,color='r', ls=':')

In [ ]:
print(StarStim_time-start_time)
print(StarStim_time-end_time)
print(StarStim_time-modified_time)
print('\n', end_time-start_time)

In [ ]:
fres = os.stat('videos/'+fname)

In [ ]:
fres

In [ ]:
time.ctime(fres.st_mtime)

In [ ]:
fres.st_mtime-t[-1]

In [ ]:
time.ctime(fres.st_ctime)

In [ ]:
StarStim_time-(fres.st_mtime-t[-1])

In [ ]:
plt.axvline(x=10.493000030517578,color='k', ls=':')

In [ ]:
plt.axvline(x=offset,color='b', ls=':')